In [18]:
import torch
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import StandardScaler
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
import re
from num2words import num2words
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, Embedding, Concatenate, AveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import HeUniform
from keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.sequence import pad_sequences
import optuna

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\spooj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\spooj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
path = "C:\\Users\\spooj\\Downloads\\TruthSeeke 2024\\TruthSeeker2023"
os.chdir(path)

In [19]:
df_2 = pd.read_csv("./Features_For_Traditional_ML_Techniques.csv")
df_2_mod = df_2.drop(columns='embeddings')

estimate the vocabulary

In [ ]:
tweets = df_2_mod['tweet']
from tensorflow.keras.preprocessing.text import Tokenizer

tokeniser = Tokenizer()
tokeniser.fit_on_texts(tweets)
vocab_size = len(tokeniser.word_index) + 1
print(vocab_size)

169377


cleaning the tweets dataset

removing stopwords

In [22]:
def remove_stopword(tweet):
    stop_words = set(stopwords.words('english'))
    filtered_tweet = [word for word in tweet.split() if word not in stop_words]
    filtered_tweet = " ".join(filtered_tweet)
    return filtered_tweet

tweets = df_2_mod['tweet']
filtered_tweets = list(map(remove_stopword, tweets))

In [ ]:
tokeniser = Tokenizer()
tokeniser.fit_on_texts(filtered_tweets)
vocab_size = len(tokeniser.word_index) + 1
print(vocab_size)

169363


lemmatization/stemming

In [23]:
def lemmatization(tweet):
    lemmatizer = nltk.WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tweet.split()]
    lemmatized_tweet = " ".join(lemmatized_tokens)
    return lemmatized_tweet

lemmatized_tweet = list(map(lemmatization, filtered_tweets))

In [ ]:
tokeniser = Tokenizer()
tokeniser.fit_on_texts(lemmatized_tweet)
vocab_size = len(tokeniser.word_index) + 1
print(vocab_size)

168250


replacing numbers with their word format, correcting typos

In [24]:
def replace_numbers_with_words(text):
    # Regular expression to find numbers
    return re.sub(r'\b\d+\b', lambda x: num2words(int(x.group())), text)

cleaned_tweets = list(map(replace_numbers_with_words, lemmatized_tweet))


In [ ]:
tokeniser = Tokenizer()
tokeniser.fit_on_texts(cleaned_tweets)
vocab_size = len(tokeniser.word_index) + 1
print(vocab_size)

166600


features selected using f-classif

In [ ]:
df_clean = df_2_mod[~df_2_mod['tweet'].isna()]
df_clean_filtered = df_clean[['statuses_count', 'cred', 'normalize_influence', 'mentions',
       'hashtags', 'URLs', 'unique_count', 'NORP_percentage',
       'GPE_percentage', 'PERSON_percentage', 'MONEY_percentage',
       'CARDINAL_percentage', 'Word count', 'Average word length',
       'present_verbs', 'past_verbs', 'adjectives', 'adverbs', 'pronouns',
       'TOs', 'conjunctions', 'exclamation', 'questions', 'capitals',
       'short_word_freq']]
labels = df_clean['majority_target']
labels = labels.astype(int)
labels = np.array(labels)
labels = np.reshape(labels, [-1,1])

features selected using mutual info classif

In [25]:
df_clean = df_2_mod[~df_2_mod['tweet'].isna()]
df_clean_filtered = df_clean[['followers_count', 'friends_count', 'favourites_count',
       'statuses_count', 'listed_count', 'cred', 'normalize_influence',
       'mentions', 'retweets', 'hashtags', 'URLs', 'unique_count',
       'PERSON_percentage', 'PERCENT_percentage', 'Word count',
       'Max word length', 'Average word length', 'present_verbs',
       'past_verbs', 'adjectives', 'adverbs', 'pronouns', 'TOs',
       'capitals', 'short_word_freq']]
labels = df_clean['majority_target']
labels = labels.astype(int)
labels = np.array(labels)
labels = np.reshape(labels, [-1,1])

In [26]:
scaler = StandardScaler()
X_clean = scaler.fit_transform(df_clean_filtered)

In [27]:
text_data = cleaned_tweets
# Tokenizer to convert text into sequences
tokenizer = Tokenizer(num_words= 50000)  # Adjust vocab size as needed
tokenizer.fit_on_texts(text_data)

# Convert text to sequences
text_sequences = tokenizer.texts_to_sequences(text_data)

# Pad sequences to ensure uniform length
max_text_len = 128
text_padded = pad_sequences(text_sequences, maxlen=max_text_len, padding='post')


In [28]:
initializer = HeUniform(seed = 3)

def create_truth_faker_model(embedding_dim, kernel_size_1, kernel_size_2, filters_1, filters_2, kernel_reg_1, dense_1, dense_2, dropout_1, dropout_2):
    # Input for structured ML features
    ml_input = Input(shape=(25,))
    ml_branch = Dense(dense_1, activation='relu', kernel_initializer=initializer)(ml_input)
    ml_branch = Dropout(dropout_1)(ml_branch)
    ml_branch = Dense(dense_2, activation='relu', kernel_initializer=initializer)(ml_branch)

    # Embedding layer
    text_input = Input(shape=(128,))
    text_emb = Embedding(input_dim=50000, output_dim=embedding_dim, input_length=128)(text_input)

    # CNN layers for text
    cnn_branch = Conv1D(filters=filters_1, kernel_size=kernel_size_1, activation='relu', kernel_initializer=initializer, kernel_regularizer=regularizers.L2(kernel_reg_1))(text_emb)
    cnn_branch = AveragePooling1D(pool_size=2)(cnn_branch)
    cnn_branch = Conv1D(filters=filters_2, kernel_size=kernel_size_2, activation='relu', kernel_initializer=initializer, kernel_regularizer=regularizers.L2(kernel_reg_1))(cnn_branch)
    cnn_branch = AveragePooling1D(pool_size=2)(cnn_branch)
    cnn_branch = Flatten()(cnn_branch)

    # Combine both branches (ML features and text features)
    combined = Concatenate()([ml_branch, cnn_branch])

    # Fully connected layers after combining
    combined = Dense(int(dense_1 / 2), activation='relu', kernel_initializer=initializer)(combined)
    combined = Dropout(dropout_2)(combined)
    combined = Dense(int(dense_2/ 2), activation='relu', kernel_initializer=initializer)(combined)

    # Output layer for classification (assuming binary classification: Fake or Real)
    output = Dense(1, activation='sigmoid')(combined)  # Use 'softmax' for multi-class

    # Create and compile the model
    model = Model(inputs=[ml_input, text_input], outputs=output)

    return model


In [29]:
X_ml_train, X_ml_test, X_text_train, X_text_test, y_train, y_test = train_test_split(X_clean, text_padded, labels, test_size=0.2, random_state=42)


In [30]:
X_ml_train, X_ml_val, X_text_train, X_text_val, y_train, y_val = train_test_split(X_ml_train, X_text_train, y_train, test_size=0.2, random_state = 42)

In [40]:
import optuna
from tensorflow.keras.optimizers import Adam

def objective(trial, X_ml_train, X_ml_val, X_text_train, X_text_val, y_train, y_val):
    # Define the search space for hyperparameters
    embedding_dim = trial.suggest_categorical("embedding_dim", [64, 128, 256])  # Embedding dimension
    filters_1 = trial.suggest_categorical("filters_1", [5, 10, 20])  # Number of filters
    filters_2 = trial.suggest_categorical("filters_2", [5, 10, 20])  # Number of filters
    kernel_size_1 = trial.suggest_categorical("kernel_size1", [5, 7, 15])  # First convolution kernel size
    kernel_size_2 = trial.suggest_categorical("kernel_size_2", [3, 5, 7])  # Second convolution kernel size
    kernel_reg_1 = trial.suggest_categorical("kernel_reg_1", [1e-2, 1e-3])  # Kernel regularization
    dense_1 = trial.suggest_categorical("dense_1", [64, 128])
    dense_2 = trial.suggest_categorical("dense_2", [32, 64])
    dropout_1 = trial.suggest_categorical("dropout_1", [0.6, 0.4])  # Dropout rate
    dropout_2 = trial.suggest_categorical("dropout_2", [0.5, 0.4])  # Dropout rate
    learning_rate = trial.suggest_categorical("learning_rate", [0.001, 0.0001, 0.00015])  # Learning rate
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])  # Batch size
    epochs = trial.suggest_categorical("epochs", [5, 10, 20])  # Number of epochs

    model = create_truth_faker_model(embedding_dim, kernel_size_1, kernel_size_2, filters_1, filters_2, kernel_reg_1, dense_1, dense_2, dropout_1, dropout_2)

    # Compile the model with the suggested learning rate
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
    checkpoint = ModelCheckpoint(
    f"../../dsa4266/cnn-{trial.number}.keras",
    monitor="val_accuracy",
    verbose=0,
    save_best_only=True,
    )
    # Train the model
    history = model.fit(
        [X_ml_train, X_text_train],  # Adjust as per your training data
        y_train,
        validation_data=([X_ml_val, X_text_val], y_val),  # Adjust as per your validation data
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stopping, checkpoint],
        verbose=1
    )

    # Return the best validation accuracy
    return max(history.history['val_accuracy'])


In [38]:
study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: objective(trial, X_ml_train, X_ml_val, X_text_train, X_text_val, y_train, y_val), n_trials=100)  # Adjust number of trials as needed

# Print best hyperparameters
print("Best hyperparameters:", study.best_params)
print("Best validation accuracy:", study.best_value)


[I 2024-10-31 22:20:15,069] A new study created in memory with name: no-name-16172305-0fcb-451f-a85f-27f8ddfae5e6
c:\Users\spooj\Downloads\dsa4266\fake_news\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5


c:\Users\spooj\Downloads\dsa4266\fake_news\lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_160', 'keras_tensor_164']. Received: the structure of inputs=('*', '*')
  warnings.warn(


5368/5368 ━━━━━━━━━━━━━━━━━━━━ 177s 32ms/step - accuracy: 0.5576 - loss: 0.7849 - val_accuracy: 0.8870 - val_loss: 0.3492
Epoch 2/5
5368/5368 ━━━━━━━━━━━━━━━━━━━━ 237s 39ms/step - accuracy: 0.9053 - loss: 0.3123 - val_accuracy: 0.9266 - val_loss: 0.2576
Epoch 3/5
5368/5368 ━━━━━━━━━━━━━━━━━━━━ 262s 39ms/step - accuracy: 0.9360 - loss: 0.2382 - val_accuracy: 0.9307 - val_loss: 0.2470
Epoch 4/5
5368/5368 ━━━━━━━━━━━━━━━━━━━━ 208s 39ms/step - accuracy: 0.9459 - loss: 0.2098 - val_accuracy: 0.9296 - val_loss: 0.2465
Epoch 5/5
5368/5368 ━━━━━━━━━━━━━━━━━━━━ 208s 39ms/step - accuracy: 0.9482 - loss: 0.2004 - val_accuracy: 0.9284 - val_loss: 0.2519


[I 2024-10-31 22:38:28,104] Trial 0 finished with value: 0.930700421333313 and parameters: {'embedding_dim': 128, 'filters_1': 10, 'filters_2': 10, 'kernel_size1': 5, 'kernel_size_2': 5, 'kernel_reg_1': 0.01, 'dense_1': 128, 'dense_2': 32, 'dropout_1': 0.4, 'dropout_2': 0.4, 'learning_rate': 0.00015, 'batch_size': 16, 'epochs': 5}. Best is trial 0 with value: 0.930700421333313.


...

1342/1342 ━━━━━━━━━━━━━━━━━━━━ 4013s 3s/step - accuracy: 0.7757 - loss: 0.4696 - val_accuracy: 0.9273 - val_loss: 0.2504
Epoch 2/10
1342/1342 ━━━━━━━━━━━━━━━━━━━━ 107s 79ms/step - accuracy: 0.9422 - loss: 0.2157 - val_accuracy: 0.9305 - val_loss: 0.2471
Epoch 3/10
1342/1342 ━━━━━━━━━━━━━━━━━━━━ 104s 78ms/step - accuracy: 0.9534 - loss: 0.1775 - val_accuracy: 0.9217 - val_loss: 0.2640
Epoch 4/10
1342/1342 ━━━━━━━━━━━━━━━━━━━━ 100s 75ms/step - accuracy: 0.9609 - loss: 0.1468 - val_accuracy: 0.9086 - val_loss: 0.3121
Epoch 5/10
1342/1342 ━━━━━━━━━━━━━━━━━━━━ 94s 70ms/step - accuracy: 0.9696 - loss: 0.1125 - val_accuracy: 0.9032 - val_loss: 0.3382


[I 2024-11-01 18:44:08,638] Trial 45 finished with value: 0.9304676055908203 and parameters: {'embedding_dim': 128, 'filters_1': 20, 'filters_2': 20, 'kernel_size1': 15, 'kernel_size_2': 5, 'kernel_reg_1': 0.001, 'dense_1': 128, 'dense_2': 64, 'dropout_1': 0.4, 'dropout_2': 0.5, 'learning_rate': 0.001, 'batch_size': 64, 'epochs': 10}. Best is trial 34 with value: 0.9352179765701294.


Epoch 1/5


c:\Users\spooj\Downloads\dsa4266\fake_news\lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_896', 'keras_tensor_900']. Received: the structure of inputs=('*', '*')
  warnings.warn(


1342/1342 ━━━━━━━━━━━━━━━━━━━━ 103s 74ms/step - accuracy: 0.5916 - loss: 0.6930 - val_accuracy: 0.9218 - val_loss: 0.2824
Epoch 2/5
1342/1342 ━━━━━━━━━━━━━━━━━━━━ 101s 75ms/step - accuracy: 0.9282 - loss: 0.2731 - val_accuracy: 0.9287 - val_loss: 0.2581
Epoch 3/5
1342/1342 ━━━━━━━━━━━━━━━━━━━━ 97s 72ms/step - accuracy: 0.9427 - loss: 0.2294 - val_accuracy: 0.9267 - val_loss: 0.2580
Epoch 4/5
 451/1342 ━━━━━━━━━━━━━━━━━━━━ 1:01 69ms/step - accuracy: 0.9473 - loss: 0.2103

In [ ]:
model = create_truth_faker_model(num_ml_features=X_ml_train.shape[1],
                                 max_text_len=X_text_train.shape[1],
                                 vocab_size=10000, embedding_dim=256,
                                 num_labels=1)




early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(filepath='./best_model_mutual_info_3.keras',  # Save the model to this file
                             monitor='val_loss',       # Monitor the validation loss
                             save_best_only=True,      # Save only the best model (based on monitored metric)
                             mode='min',               # 'min' because we want the minimum val_loss
                             verbose=1)
model.fit([X_ml_train, X_text_train], y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks = [early_stopping, checkpoint])

Epoch 1/10


c:\Users\spooj\Downloads\dsa4266\fake_news\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
c:\Users\spooj\Downloads\dsa4266\fake_news\lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_16', 'keras_tensor_20']. Received: the structure of inputs=('*', '*')
  warnings.warn(


2684/2684 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7724 - loss: 0.4710
Epoch 1: val_loss improved from inf to 0.26638, saving model to ./best_model_mutual_info_3.keras
2684/2684 ━━━━━━━━━━━━━━━━━━━━ 43s 15ms/step - accuracy: 0.7725 - loss: 0.4709 - val_accuracy: 0.9203 - val_loss: 0.2664
Epoch 2/10
2684/2684 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9269 - loss: 0.2510
Epoch 2: val_loss improved from 0.26638 to 0.26367, saving model to ./best_model_mutual_info_3.keras
2684/2684 ━━━━━━━━━━━━━━━━━━━━ 40s 15ms/step - accuracy: 0.9269 - loss: 0.2510 - val_accuracy: 0.9206 - val_loss: 0.2637
Epoch 3/10
2682/2684 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9368 - loss: 0.2217
Epoch 3: val_loss did not improve from 0.26367
2684/2684 ━━━━━━━━━━━━━━━━━━━━ 41s 15ms/step - accuracy: 0.9368 - loss: 0.2218 - val_accuracy: 0.9180 - val_loss: 0.2752
Epoch 4/10
2684/2684 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9433 - loss: 0.2064
Epoch 4: val_loss did not improve from 0.26367


In [ ]:
predictions = model.predict([X_ml_test, X_text_test])
predictions = np.where(predictions >= 0.5, 1, 0)
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

839/839 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
              precision    recall  f1-score   support

           0       0.93      0.91      0.92     13059
           1       0.91      0.94      0.93     13781

    accuracy                           0.92     26840
   macro avg       0.92      0.92      0.92     26840
weighted avg       0.92      0.92      0.92     26840

[[11854  1205]
 [  867 12914]]


In [ ]:

predictions = model.predict([X_ml_test, X_text_test])
predictions = np.where(predictions >= 0.5, 1, 0)
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

839/839 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
              precision    recall  f1-score   support

           0       0.61      0.64      0.62     13059
           1       0.64      0.61      0.63     13781

    accuracy                           0.63     26840
   macro avg       0.63      0.63      0.63     26840
weighted avg       0.63      0.63      0.63     26840

[[8360 4699]
 [5343 8438]]


In [ ]:
model = create_truth_faker_model(num_ml_features=X_ml_train.shape[1],
                                 max_text_len=X_text_train.shape[1],
                                 vocab_size=10000, embedding_dim=128,
                                 num_labels=1)


# Define EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(filepath='best_model_mutual_info.keras',  # Save the model to this file
                             monitor='val_loss',       # Monitor the validation loss
                             save_best_only=True,      # Save only the best model (based on monitored metric)
                             mode='min',               # 'min' because we want the minimum val_loss
                             verbose=1)
model.fit([X_ml_train, X_text_train], y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks = [early_stopping, checkpoint])

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


2684/2684 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8360 - loss: 0.3716
Epoch 1: val_loss improved from inf to 0.22381, saving model to best_model_mutual_info.keras
2684/2684 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - accuracy: 0.8360 - loss: 0.3716 - val_accuracy: 0.9392 - val_loss: 0.2238
Epoch 2/10
2669/2684 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9466 - loss: 0.1977
Epoch 2: val_loss did not improve from 0.22381
2684/2684 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.9466 - loss: 0.1977 - val_accuracy: 0.9362 - val_loss: 0.2259
Epoch 3/10
2676/2684 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9547 - loss: 0.1639
Epoch 3: val_loss did not improve from 0.22381
2684/2684 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9547 - loss: 0.1639 - val_accuracy: 0.9332 - val_loss: 0.2353
Epoch 4/10
2681/2684 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9620 - loss: 0.1281
Epoch 4: val_loss did not improve from 0.22381
2684/2684 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.9620 - l

In [ ]:
predictions = model.predict([X_ml_test, X_text_test])
predictions = np.where(predictions >= 0.5, 1, 0)
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

839/839 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
              precision    recall  f1-score   support

           0       0.94      0.94      0.94     13059
           1       0.94      0.94      0.94     13781

    accuracy                           0.94     26840
   macro avg       0.94      0.94      0.94     26840
weighted avg       0.94      0.94      0.94     26840

[[12232   827]
 [  810 12971]]


In [ ]:
model = create_truth_faker_model(num_ml_features=X_ml_train.shape[1],
                                 max_text_len=X_text_train.shape[1],
                                 vocab_size=10000, embedding_dim=128,
                                 num_labels=1)



# Define EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(filepath='best_model_1.keras',  # Save the model to this file
                             monitor='val_loss',       # Monitor the validation loss
                             save_best_only=True,      # Save only the best model (based on monitored metric)
                             mode='min',               # 'min' because we want the minimum val_loss
                             verbose=1)
model.fit([X_ml_train, X_text_train], y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks = [early_stopping, checkpoint])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
2684/2684 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.8358 - loss: 0.3707
Epoch 1: val_loss improved from inf to 0.22299, saving model to best_model_1.keras
2684/2684 ━━━━━━━━━━━━━━━━━━━━ 143s 52ms/step - accuracy: 0.8358 - loss: 0.3706 - val_accuracy: 0.9371 - val_loss: 0.2230
Epoch 2/10
2683/2684 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.9466 - loss: 0.1981
Epoch 2: val_loss did not improve from 0.22299
2684/2684 ━━━━━━━━━━━━━━━━━━━━ 141s 52ms/step - accuracy: 0.9466 - loss: 0.1981 - val_accuracy: 0.9373 - val_loss: 0.2261
Epoch 3/10
2684/2684 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.9541 - loss: 0.1656
Epoch 3: val_loss did not improve from 0.22299
2684/2684 ━━━━━━━━━━━━━━━━━━━━ 141s 52ms/step - accuracy: 0.9541 - loss: 0.1656 - val_accuracy: 0.9278 - val_loss: 0.2406
Epoch 4/10
2683/2684 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.9636 - loss: 0.1252
Epoch 4: val_loss did not improve from 0.22299
2684/2684 ━━━━━━━━━━━━━━━━━━━━ 142s 52ms/step - accura

In [ ]:
predictions = model.predict([X_ml_test, X_text_test])
predictions = np.where(predictions >= 0.5, 1, 0)
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

839/839 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step
              precision    recall  f1-score   support

           0       0.93      0.94      0.94     13059
           1       0.95      0.93      0.94     13781

    accuracy                           0.94     26840
   macro avg       0.94      0.94      0.94     26840
weighted avg       0.94      0.94      0.94     26840

[[12337   722]
 [  919 12862]]


In [ ]:
predictions = model.predict([X_ml_test, X_text_test])
predictions = np.where(predictions >= 0.5, 1, 0)
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

839/839 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step
              precision    recall  f1-score   support

           0       0.95      0.92      0.93     13059
           1       0.92      0.95      0.94     13781

    accuracy                           0.94     26840
   macro avg       0.94      0.93      0.94     26840
weighted avg       0.94      0.94      0.94     26840

[[11992  1067]
 [  671 13110]]


In [ ]:
from tensorflow.keras.models import save_model
save_model(model, 'truth_faker_model.keras')

In [ ]:


predictions = model.predict([X_ml_test, X_text_test])
# Convert probabilities to binary predictions (0 or 1)


839/839 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step


In [ ]:
predictions = np.where(predictions >= 0.5, 1, 0)
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

              precision    recall  f1-score   support

           0       0.94      0.88      0.91     13059
           1       0.90      0.94      0.92     13781

    accuracy                           0.91     26840
   macro avg       0.92      0.91      0.91     26840
weighted avg       0.92      0.91      0.91     26840

[[11540  1519]
 [  784 12997]]
